In [1]:
# Importing important libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
# Dineout Scrapping


rest_list = []
for page in range(1,18):
    print(f'getting page, {page}')

    s = requests.Session()

    url = f"https://www.dineout.co.in/delhi-restaurants?search_str=biryani&p={page}"  # URL of the website
    header = {'User-Agent':'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'} # Temporary user agent
    r = s.get(url, headers=header)
    soup = BeautifulSoup(r.content, 'html.parser')

    divs = soup.find_all('div', class_ = 'restnt-card restaurant')

    for item in divs:
        code = item.find('a')['href'].split('-')[-1] # restaurant code
        print(f'Getting details for {code}')
        data = s.get(f'https://www.dineout.co.in/get_rdp_data_main/delhi/{code}/restaurant_detail_main').json()

        info = data['header']
        info.pop('share') #clean up csv
        info.pop('options')
        rest_list.append(info)

df = pd.DataFrame(rest_list)

In [3]:
del_bir = df[['restaurantId','restaurantName', 'localityName', 'costForTwo', 'avgRating','ratingCount','reviewCount', 'address']].reset_index(drop=True)
dineout_scrap = del_bir[del_bir['ratingCount'] > 5]

In [17]:
# Magicpin Scraping code


url = "https://magicpin.in/Delhi/search/?dist=10&query=biriyani&rt=3"  # URL of the website 
header = {'User-Agent':'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'} # Temporary user agent
r = requests.get(url, headers=header)
soup = BeautifulSoup(r.content, 'html.parser')
rest_list=[]

divs = soup.find_all('div', class_ ="merchant-card-single")

for item in divs:
    title = item.find('h3').text.strip() # restaurant name
    loc = item.find('h4', class_ ="merchant-location").text.strip() # restaurant location
    try: # used this try and except method because some restaurants are unrated and while scrpaping those we would run into an error
        rating = item.find('span', class_="rating-value").text 
        
        
    except:
        rating = None

    price = item.find('div', class_="cft-timing").text.strip()
    price = re.sub("[^0-9]", "", price) # price for biriyani

    url1 = item.find('a').get('href')
    r2 = requests.get(url1, headers=header)
    soup1 = BeautifulSoup(r2.content, 'html.parser')
    divs1 = soup1.find_all('div', class_="ratings-header")
    for i in divs1:
        nor = i.find('span', class_="claims-title").text.strip()
        no_of_rating = re.sub("[^0-9]", "", nor)

    

    

    biry_del = {
        'name': title,
        'location': loc,
        'rating': rating,
        'numberRating': no_of_rating,
        'price': price
    }
    rest_list.append(biry_del)

magicpin_scrap = pd.DataFrame(rest_list)

In [20]:
#Cleaning the Location column

a = 0
for i in magicpin_scrap['location']:
    magicpin_scrap['location'][a] = i.split('\n', 1)[0]
    a = a+1

In [ ]:
magicpin_scrap

In [ ]:
magicpin_scrap.to_csv("magicpin_scrape.csv")
dineout_scrap.to_csv("dineout_scrape.csv")